# EO4SD SHORELINE CHANGE MAPPING AND FORECASTING

This code has been modifed by Carpenter (2020) for the project Earth Observation for Sustainable Development. Below demonstrates an example processing workflow for Benin and Togo's Coastline between 2000-2020.

This software is based on scripts and code developed by:
* Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

It enables the users to extract time-series of shoreline change over the last 20+ years at their site of interest.
There are three main steps:
1. Retrieval of median composite satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution

## Initial settings

Refer to the Set-up and Installation section of the User Handbook for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. See original methodology via https://github.com/kvos/CoastSat

In [ ]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from utils.print_utils import printProgress, printSuccess, printWarning
from coastsat import NOC_tools, NOC_shoreline, NOC_download, SDS_tools
        
# directory where the data will be accessed and stored
data_partition = 'c:\\data\\coastsat'
country = 'STV'
base_dir_path = os.path.join(data_partition, country)  
sites_dir_path = os.path.join(base_dir_path, 'sites')
results_dir_path = os.path.join(base_dir_path, 'results')
sites = os.listdir(sites_dir_path)

settings ={'results_dir_path': results_dir_path,
           'output_epsg': 32620,
           'sat_name': 'S2',

           'cloud_thresh':0.9,
           
            # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
            'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
            'buffer_size': 150,         # radius (in metres) for buffer around sandy pixels considered in the shoreline detection
            'min_length_shoreline': 200,       # minimum length (in metres) of shoreline perimeter to be valid
            'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
 
            ## Image download Parameters\n",
            # Landsat\n",
            'LCloudScore': 15,         # Mean cloud score threshold (include images with less then threshold)\n",
            'add_L7_to_L5': True,      # Add Landsat 7 to Landsat 5 median composite if they are in same time period\n",
            'add_L5_to_L7': True,      # Add Landsat 5 to Landsat 7 median composite if they are in same time period\n",
            'add_L7_to_L8': False,      # Add Landsat 7 to Landsat 8 median composite if they are in same time period\n",
            'LCloudThreshold': 35,     # Pixels from a single image in a collection larger than this cloud score threshold\n",
                                       # will be masked.\n",
           
           'coregistration': False,
           
            # Sentinel
            'CLOUD_FILTER': 50,         # [Integer] Maximum image cloud cover percent allowed in image collection'
            'CLD_PRB_THRESH': 25,       # {Integer] Cloud probability (%); values greater than are considered cloud
            'NIR_DRK_THRESH': 0.08,     # [Float] Near-infrared reflectance; values less than are considered potential cloud shadow
            'CLD_PRJ_DIST': 2,          # [Float] Maximum distance (km) to search for cloud shadows from cloud edges |
            'BUFFER': 50,               # [Integer] Distance (m) to dilate the edge of cloud-identified objects |
   
           # labels for classes for the classifier and colours for display
           'classes':{'hard': [1, [1, 0, 0]], 'nature': [2, [0, 1, 0]],
                      'urban': [3, [1, 1, 0]], 'white-water': [4, [1, 0, 1]],
                      'water': [5, [0, 0.4, 1]], 'sand': [6, [1, 0.6, 0]]},
           
            'bands': {'L8': {'pan': [['B8'],15],
                              'ms': [['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'BQA'], 30]},
                      'S2': {'10m': [['B2', 'B3', 'B4', 'B8'], 10],
                             '20m': [['B5', 'B6', 'B7', 'B8A', 'B11', 'B12'], 20],
                             '60m': [['B12'], 60]}, },
            'SWIR': 'B12',
          }

dates = [
         ['2016-01-01', '2016-12-31'],
         ['2017-01-01', '2017-12-31'],
         ['2018-01-01', '2018-12-31'],
         ['2019-01-01', '2019-12-31'],
         ['2020-01-01', '2020-12-31'],
                                     ]

## download optical data from GEE

In [ ]:
"""
sites_rois = [[[-61.166703546,13.386285093], [-61.166703546,13.349894905],
               [-61.216490795,13.386285093], [-61.216490795,13.349894905]],
              [[-61.2057165049999,13.3550601210001], [-61.2057165049999,13.284226463],
               [-61.253194809,13.3550601210001], [-61.253194809,13.284226463]],
              [[-61.252042691,13.2913084200001], [-61.252042691,13.230160147],
               [-61.28130129,13.2913084200001], [-61.28130129,13.230160147]],
              [[-61.259699901,13.233287525], [-61.259699901,13.1776702440001],
               [-61.2866043509999,13.233287525], [-61.2866043509999,13.1776702440001]],
              [[-61.208049102,13.1832541430001], [-61.208049102,13.13343525],
               [-61.2678437329999,13.1832541430001], [-61.2678437329999,13.13343525]],
              [[-61.1270383959999,13.3867923610001], [-61.1270383959999,13.365715129],
               [-61.1728832279999,13.3867923610001], [-61.1728832279999,13.365715129]],
              [[-61.107290701,13.3673933800001],  [-61.107290701,13.2866351280001],
               [-61.1352551479999,13.3673933800001], [-61.1352551479999,13.2866351280001]],
              [[-61.1069099169999,13.29456651], [-61.1069099169999,13.2067607450001],
               [-61.1298686229999,13.29456651], [-61.1298686229999,13.2067607450001]],
              [[-61.125427504,13.2127523570001], [-61.125427504,13.137030705],
               [-61.1617257629999,13.2127523570001], [-61.1617257629999,13.137030705]],
              [[-61.157544898,13.142036698],[-61.157544898,13.1203589470001],
               [-61.2120815659999,13.142036698],[-61.2120815659999,13.1203589470001]],
              [[-61.38028666,12.651823393],[-61.38028666,12.627519251],
               [-61.4029449619999,12.651823393],[-61.4029449619999,12.627519251]]]"""

#for site_index, site_roi in enumerate(sites_rois):
for site in sites:       
    kml_filepath = os.path.join(sites_dir_path, site)
    kml_polygon = NOC_tools.polygon_from_kml(kml_filepath)
    roi_polygon = SDS_tools.smallest_rectangle(kml_polygon)
    roi_polygon = SDS_tools.smallest_rectangle(site_roi)
#    roi_polygon = site_roi
#    site_name = 'site_' + str(site_index+1)

    site_name = site[:site.find('.')]
    site_dir_path = os.path.join(base_dir_path, site_name)
#    if not os.path.exists(site_dir_path):
#        os.mkdir(site_dir_path)
    median_dir_path = os.path.join(site_dir_path, 'median')
    if not os.path.exists(median_dir_path):
        os.mkdir(median_dir_path)
        
    settings['median_dir_path'] = median_dir_path
    settings['polygon'] = roi_polygon
    settings['site_name'] = site_name
        
    for date_pair in dates:
        
        settings['dates'] = date_pair 
        
        printProgress(f'processing {site_name}: {date_pair}')
        
        NOC_download.retrieve_median_optical(settings)
    
    NOC_download.save_metadata(settings)
            
print('FINISHED ...')

## find reference shoreline

In [ ]:
settings['date_range'] = ['2016-01-01', '2020-12-31']
settings['pansharpen'] = True
settings['max_dist_ref'] = 75
    
for site in sites:
    
    site_name = site[:site.find('.')]
    median_dir_path = os.path.join(base_dir_path, site_name, 'median')
    settings['site_name'] = site_name
    settings['median_dir_path'] = median_dir_path
    
    printProgress(f'processing {site_name}')
    
    %matplotlib qt
    NOC_shoreline.find_reference_shoreline(settings) 

print('FINISHED ...')

## extract shorelines

In [ ]:
settings['date_range'] = ['2016-01-01', '2020-12-31']

"""
sites_rois = [[[-61.166703546,13.386285093], [-61.166703546,13.349894905],
               [-61.216490795,13.386285093], [-61.216490795,13.349894905]],
              [[-61.2057165049999,13.3550601210001], [-61.2057165049999,13.284226463],
               [-61.253194809,13.3550601210001], [-61.253194809,13.284226463]],
              [[-61.252042691,13.2913084200001], [-61.252042691,13.230160147],
               [-61.28130129,13.2913084200001], [-61.28130129,13.230160147]],
              [[-61.259699901,13.233287525], [-61.259699901,13.1776702440001],
               [-61.2866043509999,13.233287525], [-61.2866043509999,13.1776702440001]],
              [[-61.208049102,13.1832541430001], [-61.208049102,13.13343525],
               [-61.2678437329999,13.1832541430001], [-61.2678437329999,13.13343525]],
              [[-61.1270383959999,13.3867923610001], [-61.1270383959999,13.365715129],
               [-61.1728832279999,13.3867923610001], [-61.1728832279999,13.365715129]],
              [[-61.107290701,13.3673933800001],  [-61.107290701,13.2866351280001],
               [-61.1352551479999,13.3673933800001], [-61.1352551479999,13.2866351280001]],
              [[-61.1069099169999,13.29456651], [-61.1069099169999,13.2067607450001],
               [-61.1298686229999,13.29456651], [-61.1298686229999,13.2067607450001]],
              [[-61.125427504,13.2127523570001], [-61.125427504,13.137030705],
               [-61.1617257629999,13.2127523570001], [-61.1617257629999,13.137030705]],
              [[-61.157544898,13.142036698],[-61.157544898,13.1203589470001],
               [-61.2120815659999,13.142036698],[-61.2120815659999,13.1203589470001]],
              [[-61.38028666,12.651823393],[-61.38028666,12.627519251],
               [-61.4029449619999,12.651823393],[-61.4029449619999,12.627519251]]]"""


settings['pansharpen'] = True
settings['reference_threshold'] = -0.475
settings['max_dist_ref'] = 100


for site in sites:
    
    site_name = site[:site.find('.')]

    median_dir_path = os.path.join(base_dir_path, site_name, 'median')
    settings['site_name'] = site_name
    settings['median_dir_path'] = median_dir_path
    printProgress('')
        
    for date_pair in dates:
 
        settings['dates'] = date_pair 
    
        printProgress(f'processing {site_name}: {date_pair}')
        
        %matplotlib qt
        try:
            NOC_shoreline.extract_shoreline_optical(settings, batch=True)
        except Exception as inst:
            printWarning('skipping image')
            print(type(inst))
            print(inst.args)
                
print('FINISHED ...')